In [69]:
import os
import pymorphy2
import re
import string
import numpy as np
from collections import Counter

In [2]:
#Парсинг датасета

In [3]:
dataset = 'devset'

In [4]:
factrueval_dev_tokens = dict()
factrueval_dev_tokens_list = []
factrueval_dev_spans = dict()
factrueval_dev_objects = dict()

In [5]:
#Проходимся по всем файлам в папке
for file in os.listdir('./'+ dataset + '/'):
    if file.endswith('tokens'):
        with open('./' + dataset + '/' + file, 'r+', encoding='utf-8') as file_obj: 
            for line in file_obj.readlines():
                token = line.split()
                if token != []:
                     factrueval_dev_tokens[int(token[0])] = token[1:]
                factrueval_dev_tokens_list.append(token)
                
    if file.endswith('spans'):
        with open('./' + dataset + '/' + file, 'r+', encoding='utf-8') as file_obj:
            for line in file_obj.readlines():
                span = line.split()
                if span != []:
                    factrueval_dev_spans[span[0]] = span[1:]
    
    if file.endswith('objects'):
        with open('./' + dataset + '/' + file, 'r+', encoding='utf-8') as file_obj:
            for line in file_obj.readlines():
                object = line.split('#')[0].split()
                if object != []:
                    factrueval_dev_objects[object[0]] = object[1:]

In [6]:
factrueval_dev_tokens

{143783: ['0', '1', 'В'],
 143784: ['2', '11', 'понедельник'],
 143785: ['14', '2', '28'],
 143786: ['17', '4', 'июня'],
 143787: ['22', '1', 'у'],
 143788: ['24', '6', 'здания'],
 143789: ['31', '5', 'мэрии'],
 143790: ['37', '6', 'Москвы'],
 143791: ['44', '2', 'на'],
 143792: ['47', '8', 'Тверской'],
 143793: ['56', '7', 'площади'],
 143794: ['64', '10', 'состоялась'],
 143795: ['75', '9', 'очередная'],
 143796: ['85', '19', 'несанкционированная'],
 143797: ['105', '5', 'акция'],
 143798: ['111', '8', 'протеста'],
 143799: ['120', '1', '«'],
 143800: ['121', '4', 'День'],
 143801: ['126', '5', 'гнева'],
 143802: ['131', '1', '»'],
 143803: ['132', '1', ','],
 143804: ['134', '1', 'в'],
 143805: ['136', '4', 'этот'],
 143806: ['141', '3', 'раз'],
 143807: ['145', '12', 'направленная'],
 143808: ['157', '1', ','],
 143809: ['159', '7', 'главным'],
 143810: ['167', '7', 'образом'],
 143811: ['174', '1', ','],
 143812: ['176', '6', 'против'],
 143813: ['183', '8', 'политики'],
 143814: 

In [7]:
factrueval_dev_tokens_list

[['143783', '0', '1', 'В'],
 ['143784', '2', '11', 'понедельник'],
 ['143785', '14', '2', '28'],
 ['143786', '17', '4', 'июня'],
 ['143787', '22', '1', 'у'],
 ['143788', '24', '6', 'здания'],
 ['143789', '31', '5', 'мэрии'],
 ['143790', '37', '6', 'Москвы'],
 ['143791', '44', '2', 'на'],
 ['143792', '47', '8', 'Тверской'],
 ['143793', '56', '7', 'площади'],
 ['143794', '64', '10', 'состоялась'],
 ['143795', '75', '9', 'очередная'],
 ['143796', '85', '19', 'несанкционированная'],
 ['143797', '105', '5', 'акция'],
 ['143798', '111', '8', 'протеста'],
 ['143799', '120', '1', '«'],
 ['143800', '121', '4', 'День'],
 ['143801', '126', '5', 'гнева'],
 ['143802', '131', '1', '»'],
 ['143803', '132', '1', ','],
 ['143804', '134', '1', 'в'],
 ['143805', '136', '4', 'этот'],
 ['143806', '141', '3', 'раз'],
 ['143807', '145', '12', 'направленная'],
 ['143808', '157', '1', ','],
 ['143809', '159', '7', 'главным'],
 ['143810', '167', '7', 'образом'],
 ['143811', '174', '1', ','],
 ['143812', '176', 

In [8]:
factrueval_dev_spans

{'22763': ['loc_name', '37', '6', '143790', '1', '#', '143790', 'Москвы'],
 '22764': ['org_descr', '31', '5', '143789', '1', '#', '143789', 'мэрии'],
 '22765': ['loc_name', '47', '8', '143792', '1', '#', '143792', 'Тверской'],
 '22766': ['loc_descr', '56', '7', '143793', '1', '#', '143793', 'площади'],
 '22767': ['name', '313', '4', '143831', '1', '#', '143831', 'Юрия'],
 '22768': ['surname', '318', '7', '143832', '1', '#', '143832', 'Лужкова'],
 '22769': ['loc_name', '306', '6', '143830', '1', '#', '143830', 'Москвы'],
 '22770': ['loc_name', '477', '6', '143853', '1', '#', '143853', 'Москве'],
 '22771': ['loc_name', '531', '6', '143862', '1', '#', '143862', 'России'],
 '22772': ['org_name',
  '562',
  '10',
  '143868',
  '1',
  '#',
  '143868',
  'Мосгордумы'],
 '22773': ['name', '610', '6', '143874', '1', '#', '143874', 'Бориса'],
 '22774': ['surname', '617', '7', '143875', '1', '#', '143875', 'Громова'],
 '22775': ['name', '756', '4', '143899', '1', '#', '143899', 'Юрия'],
 '22776':

In [9]:
factrueval_dev_objects

{'10433': ['Org', '22763', '22764'],
 '10547': ['LocOrg', '22763'],
 '10434': ['Location', '22765', '22766'],
 '10435': ['Person', '22767', '22768'],
 '10436': ['LocOrg', '22769'],
 '10437': ['Location', '22770'],
 '10438': ['LocOrg', '22771'],
 '10439': ['Org', '22772'],
 '10440': ['Person', '22773', '22774'],
 '10441': ['Person', '22775', '22776'],
 '40649': ['LocOrg', '68733'],
 '10443': ['Location', '22782', '22783', '22784'],
 '10444': ['Person', '22782', '22783'],
 '10445': ['Person', '22785'],
 '10447': ['Org', '22788', '22789', '22790'],
 '10454': ['LocOrg', '22788', '22789'],
 '10455': ['Org', '22793', '22794'],
 '10456': ['Org', '22793', '22794', '22795', '22808'],
 '10450': ['Person', '22796', '22797'],
 '10451': ['Person', '22798', '22799'],
 '10452': ['Org', '22800', '22801'],
 '10453': ['Person', '22802', '22803'],
 '10457': ['Org', '22809', '22810'],
 '10458': ['Org', '22811', '22812'],
 '10459': ['Org', '22813'],
 '10460': ['Org', '22814', '22815'],
 '10461': ['Org', '2

In [10]:
all_ne = []
for key, value in factrueval_dev_objects.items():
    spans = value[1:]
    ne = value[0]
    all_tokens = []
    for span in spans:
        span_obj = factrueval_dev_spans[span]
        token = int(span_obj[3])
        num_of_tokens = int(span_obj[4])
        for i in range(num_of_tokens):
            all_tokens.append(token + i)
    all_ne.append([ne, sorted(all_tokens)])

In [11]:
all_ne

[['Org', [143789, 143790]],
 ['LocOrg', [143790]],
 ['Location', [143792, 143793]],
 ['Person', [143831, 143832]],
 ['LocOrg', [143830]],
 ['Location', [143853]],
 ['LocOrg', [143862]],
 ['Org', [143868]],
 ['Person', [143874, 143875]],
 ['Person', [143899, 143900]],
 ['LocOrg', [143872]],
 ['Location', [143933, 143934, 143935]],
 ['Person', [143934, 143935]],
 ['Person', [143993]],
 ['Org', [144001, 144002, 144003, 144004]],
 ['LocOrg', [144003, 144004]],
 ['Org', [144009, 144011]],
 ['Org', [144007, 144008, 144008, 144009, 144011]],
 ['Person', [144013, 144014]],
 ['Person', [144018, 144019]],
 ['Org', [144022, 144024, 144025]],
 ['Person', [144027, 144028]],
 ['Org', [144033, 144035, 144036, 144037]],
 ['Org', [144040, 144040, 144041, 144042]],
 ['Org', [144044]],
 ['Org', [144047, 144049, 144050]],
 ['Org', [144054, 144055]],
 ['Org', [144058, 144058, 144059, 144060]],
 ['Org', [144062, 144062, 144063, 144064, 144065, 144066]],
 ['Org', [144068, 144070, 144071]],
 ['Org', [144075, 

In [12]:
for ne_tokens in all_ne:
    ne = ne_tokens[0]
    token = ne_tokens[1]
    for i in range(len(token)):
        if token[i] in factrueval_dev_tokens.keys():
            if len(token) == 1:
                factrueval_dev_tokens[token[i]].append("S-" + ne)
            elif (i == 0 and token[i + 1] - token[i] > 1) or (i == len(token) - 1 and token[i] - token[i - 1] > 1) or (token[i] - token[i - 1] > 1 and token[i + 1] - token[i] > 1):
                factrueval_dev_tokens[token[i]].append("S-" + ne)
            elif (i == 0  and token[i + 1] - token[i] == 1) or (i != len(token) - 1 and token[i] - token[i - 1] > 1 and token[i + 1] - token[i] == 1):
                factrueval_dev_tokens[token[i]].append("B-" + ne)
            elif (i == len(token) - 1 and token[i] - token[i - 1] == 1) or (i != 0 and token[i] - token[i - 1] == 1 and token[i + 1] - token[i] > 1):
                factrueval_dev_tokens[token[i]].append("E-" + ne)
            else: 
                factrueval_dev_tokens[token[i]].append("I-" + ne)

In [13]:
factrueval_dev_tokens

{143783: ['0', '1', 'В'],
 143784: ['2', '11', 'понедельник'],
 143785: ['14', '2', '28'],
 143786: ['17', '4', 'июня'],
 143787: ['22', '1', 'у'],
 143788: ['24', '6', 'здания'],
 143789: ['31', '5', 'мэрии', 'B-Org'],
 143790: ['37', '6', 'Москвы', 'E-Org', 'S-LocOrg'],
 143791: ['44', '2', 'на'],
 143792: ['47', '8', 'Тверской', 'B-Location'],
 143793: ['56', '7', 'площади', 'E-Location'],
 143794: ['64', '10', 'состоялась'],
 143795: ['75', '9', 'очередная'],
 143796: ['85', '19', 'несанкционированная'],
 143797: ['105', '5', 'акция'],
 143798: ['111', '8', 'протеста'],
 143799: ['120', '1', '«'],
 143800: ['121', '4', 'День'],
 143801: ['126', '5', 'гнева'],
 143802: ['131', '1', '»'],
 143803: ['132', '1', ','],
 143804: ['134', '1', 'в'],
 143805: ['136', '4', 'этот'],
 143806: ['141', '3', 'раз'],
 143807: ['145', '12', 'направленная'],
 143808: ['157', '1', ','],
 143809: ['159', '7', 'главным'],
 143810: ['167', '7', 'образом'],
 143811: ['174', '1', ','],
 143812: ['176', '6

In [14]:
for i in range(len(factrueval_dev_tokens_list)):
        if factrueval_dev_tokens_list[i] == []:
            continue
        number_of_token = factrueval_dev_tokens_list[i][0]
        if int(number_of_token) in factrueval_dev_tokens.keys() and len(factrueval_dev_tokens[int(number_of_token)]) >= 4:
            ne = factrueval_dev_tokens[int(number_of_token)][3]
            factrueval_dev_tokens_list[i].append(ne)
        else:
            factrueval_dev_tokens_list[i].append("O")

In [15]:
factrueval_dev_tokens_list

[['143783', '0', '1', 'В', 'O'],
 ['143784', '2', '11', 'понедельник', 'O'],
 ['143785', '14', '2', '28', 'O'],
 ['143786', '17', '4', 'июня', 'O'],
 ['143787', '22', '1', 'у', 'O'],
 ['143788', '24', '6', 'здания', 'O'],
 ['143789', '31', '5', 'мэрии', 'B-Org'],
 ['143790', '37', '6', 'Москвы', 'E-Org'],
 ['143791', '44', '2', 'на', 'O'],
 ['143792', '47', '8', 'Тверской', 'B-Location'],
 ['143793', '56', '7', 'площади', 'E-Location'],
 ['143794', '64', '10', 'состоялась', 'O'],
 ['143795', '75', '9', 'очередная', 'O'],
 ['143796', '85', '19', 'несанкционированная', 'O'],
 ['143797', '105', '5', 'акция', 'O'],
 ['143798', '111', '8', 'протеста', 'O'],
 ['143799', '120', '1', '«', 'O'],
 ['143800', '121', '4', 'День', 'O'],
 ['143801', '126', '5', 'гнева', 'O'],
 ['143802', '131', '1', '»', 'O'],
 ['143803', '132', '1', ',', 'O'],
 ['143804', '134', '1', 'в', 'O'],
 ['143805', '136', '4', 'этот', 'O'],
 ['143806', '141', '3', 'раз', 'O'],
 ['143807', '145', '12', 'направленная', 'O'],


In [16]:
final = []
for el in factrueval_dev_tokens_list:
    if el == []:
        final.append(el)
    else:
        final.append([el[3], el[1], el[2], el[4]])

In [17]:
final

[['В', '0', '1', 'O'],
 ['понедельник', '2', '11', 'O'],
 ['28', '14', '2', 'O'],
 ['июня', '17', '4', 'O'],
 ['у', '22', '1', 'O'],
 ['здания', '24', '6', 'O'],
 ['мэрии', '31', '5', 'B-Org'],
 ['Москвы', '37', '6', 'E-Org'],
 ['на', '44', '2', 'O'],
 ['Тверской', '47', '8', 'B-Location'],
 ['площади', '56', '7', 'E-Location'],
 ['состоялась', '64', '10', 'O'],
 ['очередная', '75', '9', 'O'],
 ['несанкционированная', '85', '19', 'O'],
 ['акция', '105', '5', 'O'],
 ['протеста', '111', '8', 'O'],
 ['«', '120', '1', 'O'],
 ['День', '121', '4', 'O'],
 ['гнева', '126', '5', 'O'],
 ['»', '131', '1', 'O'],
 [',', '132', '1', 'O'],
 ['в', '134', '1', 'O'],
 ['этот', '136', '4', 'O'],
 ['раз', '141', '3', 'O'],
 ['направленная', '145', '12', 'O'],
 [',', '157', '1', 'O'],
 ['главным', '159', '7', 'O'],
 ['образом', '167', '7', 'O'],
 [',', '174', '1', 'O'],
 ['против', '176', '6', 'O'],
 ['политики', '183', '8', 'O'],
 ['московских', '192', '10', 'O'],
 ['и', '203', '1', 'O'],
 ['подмосковных'

In [18]:
def prepare(dataset):
    factrueval_dev_tokens = dict()
    factrueval_dev_tokens_list = []
    factrueval_dev_spans = dict()
    factrueval_dev_objects = dict()
    for file in os.listdir('./'+ dataset + '/'):
        if file.endswith('tokens'):
            with open('./' + dataset + '/' + file, 'r+', encoding='utf-8') as file_obj:
                lines = file_obj.readlines()
                tokens = [line.rstrip().split() for line in lines if line.rstrip().split() != []]
                for token in tokens:
                    factrueval_dev_tokens[int(token[0])] = token[1:]
                tokens = [line.rstrip().split() for line in lines]
                for token in tokens:
                    factrueval_dev_tokens_list.append(token)

        if file.endswith('spans'):
            with open('./' + dataset + '/' + file, 'r+', encoding='utf-8') as file_obj:
                spans = [line.rstrip().split() for line in file_obj.readlines() if line.rstrip().split() != []]
                for span in spans:
                    factrueval_dev_spans[span[0]] = span[1:]

        if file.endswith('objects'):
            with open('./' + dataset + '/' + file, 'r+', encoding='utf-8') as file_obj:
                objects = [line.rstrip().split('#')[0].split() for line in file_obj.readlines() if line.rstrip().split() != []]
                for obj in objects:
                    factrueval_dev_objects[obj[0]] = obj[1:]

    all_ne = []
    for key, value in factrueval_dev_objects.items():
        spans = value[1:]
        ne = value[0]
        all_tokens = []
        for span in spans:
            span_obj = factrueval_dev_spans[span]
            token = int(span_obj[3])
            num_of_tokens = int(span_obj[4])
            for i in range(num_of_tokens):
                all_tokens.append(token + i)
        all_ne.append([ne, sorted(all_tokens)])

    for ne_tokens in all_ne:
        ne = ne_tokens[0]
        token = ne_tokens[1]
        for i in range(len(token)):
            if token[i] in factrueval_dev_tokens.keys():
                if len(token) == 1:
                    factrueval_dev_tokens[token[i]].append("S-" + ne)
                elif (i == 0 and token[i + 1] - token[i] > 1) or (i == len(token) - 1 and token[i] - token[i - 1] > 1) or (token[i] - token[i - 1] > 1 and token[i + 1] - token[i] > 1):
                    factrueval_dev_tokens[token[i]].append("S-" + ne)
                elif (i == 0  and token[i + 1] - token[i] == 1) or (i != len(token) - 1 and token[i] - token[i - 1] > 1 and token[i + 1] - token[i] == 1):
                    factrueval_dev_tokens[token[i]].append("B-" + ne)
                elif (i == len(token) - 1 and token[i] - token[i - 1] == 1) or (i != 0 and token[i] - token[i - 1] == 1 and token[i + 1] - token[i] > 1):
                    factrueval_dev_tokens[token[i]].append("E-" + ne)
                else: 
                    factrueval_dev_tokens[token[i]].append("I-" + ne)

    for i in range(len(factrueval_dev_tokens_list)):
        if factrueval_dev_tokens_list[i] == []:
            continue
        number_of_token = factrueval_dev_tokens_list[i][0]
        if int(number_of_token) in factrueval_dev_tokens.keys() and len(factrueval_dev_tokens[int(number_of_token)]) >= 4:
            ne = factrueval_dev_tokens[int(number_of_token)][3]
            factrueval_dev_tokens_list[i].append(ne)
        else:
            factrueval_dev_tokens_list[i].append("O")

    final = []
    for el in factrueval_dev_tokens_list:
        if el == []:
            final.append(el)
        else:
            final.append([el[3], el[1], el[2], el[4]])
    return final

In [19]:
devset = prepare('devset')

In [20]:
devset

[['В', '0', '1', 'O'],
 ['понедельник', '2', '11', 'O'],
 ['28', '14', '2', 'O'],
 ['июня', '17', '4', 'O'],
 ['у', '22', '1', 'O'],
 ['здания', '24', '6', 'O'],
 ['мэрии', '31', '5', 'B-Org'],
 ['Москвы', '37', '6', 'E-Org'],
 ['на', '44', '2', 'O'],
 ['Тверской', '47', '8', 'B-Location'],
 ['площади', '56', '7', 'E-Location'],
 ['состоялась', '64', '10', 'O'],
 ['очередная', '75', '9', 'O'],
 ['несанкционированная', '85', '19', 'O'],
 ['акция', '105', '5', 'O'],
 ['протеста', '111', '8', 'O'],
 ['«', '120', '1', 'O'],
 ['День', '121', '4', 'O'],
 ['гнева', '126', '5', 'O'],
 ['»', '131', '1', 'O'],
 [',', '132', '1', 'O'],
 ['в', '134', '1', 'O'],
 ['этот', '136', '4', 'O'],
 ['раз', '141', '3', 'O'],
 ['направленная', '145', '12', 'O'],
 [',', '157', '1', 'O'],
 ['главным', '159', '7', 'O'],
 ['образом', '167', '7', 'O'],
 [',', '174', '1', 'O'],
 ['против', '176', '6', 'O'],
 ['политики', '183', '8', 'O'],
 ['московских', '192', '10', 'O'],
 ['и', '203', '1', 'O'],
 ['подмосковных'

In [21]:
testset = prepare('testset')

In [22]:
testset

[['Александр', '0', '9', 'B-Person'],
 ['Вертинский', '10', '10', 'E-Person'],
 ['.', '20', '1', 'O'],
 ['«', '22', '1', 'O'],
 ['Я', '23', '1', 'O'],
 ['не', '25', '2', 'O'],
 ['знаю', '28', '4', 'O'],
 [',', '32', '1', 'O'],
 ['зачем', '34', '5', 'O'],
 ['и', '40', '1', 'O'],
 ['кому', '42', '4', 'O'],
 ['это', '47', '3', 'O'],
 ['нужно', '51', '5', 'O'],
 ['…', '56', '1', 'O'],
 ['»', '57', '1', 'O'],
 [],
 ['21', '60', '2', 'O'],
 ['марта', '63', '5', 'O'],
 ['1889', '69', '4', 'O'],
 ['года', '74', '4', 'O'],
 ['родился', '79', '7', 'O'],
 ['главный', '87', '7', 'O'],
 ['русский', '95', '7', 'O'],
 ['шансонье', '103', '8', 'O'],
 ['XX', '112', '2', 'O'],
 ['века', '115', '4', 'O'],
 [',', '119', '1', 'O'],
 ['печальный', '121', '9', 'O'],
 ['Пьеро', '131', '5', 'S-Person'],
 [',', '136', '1', 'O'],
 ['вписавший', '138', '9', 'O'],
 ['свою', '148', '4', 'O'],
 ['судьбу', '153', '6', 'O'],
 ['в', '160', '1', 'O'],
 ['историю', '162', '7', 'O'],
 ['отечественной', '170', '13', 'O'],


In [23]:
#Fit_transform для получения матрицы X

In [24]:
column_types = None
_column_types = column_types if column_types is not None else ["WORD"]

In [25]:
context_len = 2
_context_len = context_len

In [26]:
data = []
for el in devset:
    if el:
        data.append(el[0])

In [27]:
len(data)

30940

In [28]:
data

['В',
 'понедельник',
 '28',
 'июня',
 'у',
 'здания',
 'мэрии',
 'Москвы',
 'на',
 'Тверской',
 'площади',
 'состоялась',
 'очередная',
 'несанкционированная',
 'акция',
 'протеста',
 '«',
 'День',
 'гнева',
 '»',
 ',',
 'в',
 'этот',
 'раз',
 'направленная',
 ',',
 'главным',
 'образом',
 ',',
 'против',
 'политики',
 'московских',
 'и',
 'подмосковных',
 'властей',
 '.',
 'Среди',
 'требований',
 ',',
 'выдвигаемых',
 'организаторами',
 'акции',
 ':',
 '«',
 'немедленная',
 'отставка',
 'мэра',
 'Москвы',
 'Юрия',
 'Лужкова',
 ',',
 'расследование',
 'итогов',
 'его',
 'деятельности',
 '»',
 ',',
 '«',
 'созыв',
 'московского',
 'общественного',
 'форума',
 'для',
 'обсуждения',
 'путей',
 'реформирования',
 'основных',
 'сфер',
 'жизнедеятельности',
 'в',
 'Москве',
 '»',
 ',',
 '«',
 'восстановление',
 'прямых',
 'выборов',
 'глав',
 'регионов',
 'России',
 '»',
 ',',
 '«',
 'роспуск',
 'нелегитимной',
 'Мосгордумы',
 '»',
 ',',
 'отставка',
 'подмосковного',
 'губернатора',
 'Бор

In [29]:
# ЗАЧЕМ?
# Добавляем пустые "слова" в начало и конец (для контекста) #
data = [["" for i in range(len(_column_types))] for i in range(_context_len)] + data
data = data + [["" for i in range(len(_column_types))] for i in range(_context_len)]

In [30]:
len(data)

30944

In [31]:
data

[[''],
 [''],
 'В',
 'понедельник',
 '28',
 'июня',
 'у',
 'здания',
 'мэрии',
 'Москвы',
 'на',
 'Тверской',
 'площади',
 'состоялась',
 'очередная',
 'несанкционированная',
 'акция',
 'протеста',
 '«',
 'День',
 'гнева',
 '»',
 ',',
 'в',
 'этот',
 'раз',
 'направленная',
 ',',
 'главным',
 'образом',
 ',',
 'против',
 'политики',
 'московских',
 'и',
 'подмосковных',
 'властей',
 '.',
 'Среди',
 'требований',
 ',',
 'выдвигаемых',
 'организаторами',
 'акции',
 ':',
 '«',
 'немедленная',
 'отставка',
 'мэра',
 'Москвы',
 'Юрия',
 'Лужкова',
 ',',
 'расследование',
 'итогов',
 'его',
 'деятельности',
 '»',
 ',',
 '«',
 'созыв',
 'московского',
 'общественного',
 'форума',
 'для',
 'обсуждения',
 'путей',
 'реформирования',
 'основных',
 'сфер',
 'жизнедеятельности',
 'в',
 'Москве',
 '»',
 ',',
 '«',
 'восстановление',
 'прямых',
 'выборов',
 'глав',
 'регионов',
 'России',
 '»',
 ',',
 '«',
 'роспуск',
 'нелегитимной',
 'Мосгордумы',
 '»',
 ',',
 'отставка',
 'подмосковного',
 'губер

In [32]:
# Находим индексы столбцов в переданных данных #
word_index = _column_types.index("WORD")
if "POS" in _column_types:
    pos_index = _column_types.index("POS")
else:
    pos_index = None
if "CHUNK" in _column_types:
    chunk_index = _column_types.index("CHUNK")
else:
    chunk_index = None

In [33]:
word_index

0

In [61]:
_column_types

['WORD']

In [34]:
# Список признаков (строка == набор признаков для слова из массива data) #
features_list = []

In [35]:
_morph = pymorphy2.MorphAnalyzer()

In [36]:
_morph.parse('Открой')[0].tag.POS

'VERB'

In [43]:
# Возвращает POS-тег слова #
def get_pos_tag(token):
    res = _morph.parse(token)[0].tag.POS
    if res:
        return res
    return None

In [38]:
def get_capital(token):
    pattern = re.compile("[{}]+$".format(re.escape(string.punctuation)))
    if pattern.match(token):
        return "none"
    if len(token) == 0:
        return "none"
    if token.islower():
        return "lower"
    elif token.isupper():
        return "upper"
    elif token[0].isupper() and len(token) == 1:
        return "proper"
    elif token[0].isupper() and token[1:].islower():
        return "proper"
    else:
        return "camel"

In [39]:
# Признак того, является ли слово пунктуацией #
def get_is_punct(token):
    pattern = re.compile("[{}]+$".format(re.escape(string.punctuation)))
    if pattern.match(token):
        return "yes"
    else:
        return "no"

In [40]:
# Признак того, является ли слово числом #
def get_is_number(token):
    try:
        complex(token)
    except ValueError:
        return "no"
    return "yes"

In [44]:
# Возвращает начальную форму слова #
def get_initial(token):
    res = _morph.parse(token)[0].normal_form
    if res:
        return res
    return None

In [46]:
# Заполнение массива features_list "сырыми" данными (без отсева) #
for k in range(len(data) - 2 * _context_len):
    arr = []
    i = k + _context_len

    if pos_index is not None:
        pos_arr = [data[i][pos_index]]
        for j in range(1, _context_len + 1):
            pos_arr.append(data[i - j][pos_index])
            pos_arr.append(data[i + j][pos_index])
    else:
        pos_arr = [get_pos_tag(data[i][word_index])]
        for j in range(1, _context_len + 1):
            pos_arr.append(get_pos_tag(data[i - j][word_index]))
            pos_arr.append(get_pos_tag(data[i + j][word_index]))
    arr += pos_arr

    if chunk_index is not None:
        chunk_arr = [data[i][chunk_index]]
        for j in range(1, _context_len + 1):
            chunk_arr.append(data[i - j][chunk_index])
            chunk_arr.append(data[i + j][chunk_index])
        arr += chunk_arr

    capital_arr = [get_capital(data[i][word_index])]
    for j in range(1, _context_len + 1):
        capital_arr.append(get_capital(data[i - j][word_index]))
        capital_arr.append(get_capital(data[i + j][word_index]))
    arr += capital_arr

    is_punct_arr = [get_is_punct(data[i][word_index])]
    for j in range(1, _context_len + 1):
        is_punct_arr.append(get_is_punct(data[i - j][word_index]))
        is_punct_arr.append(get_is_punct(data[i + j][word_index]))
    arr += is_punct_arr

    is_number_arr = [get_is_number(data[i][word_index])]
    for j in range(1, _context_len + 1):
        is_number_arr.append(get_is_number(data[i - j][word_index]))
        is_number_arr.append(get_is_number(data[i + j][word_index]))
    arr += is_number_arr

    initial_arr = [get_initial(data[i][word_index])]
    for j in range(1, _context_len + 1):
        initial_arr.append(get_initial(data[i - j][word_index]))
        initial_arr.append(get_initial(data[i + j][word_index]))
    arr += initial_arr

    features_list.append(arr)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113


3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732
3733
3734
3735
3736
3737
3738
3739
3740
3741
3742
3743
3744
3745
3746
3747
3748
3749
3750
3751
3752
3753
3754
3755
3756
3757
3758
3759
3760
3761
3762
3763
3764
3765
3766
3767
3768
3769
3770
3771
3772
3773
3774
3775
3776
3777
3778
3779
3780
3781
3782
3783
3784
3785
3786
3787
3788
3789
3790
3791
3792
3793
3794
3795
3796
3797
3798
3799
3800
3801
3802
3803
3804
3805
3806
3807
3808
3809
3810
3811
3812
3813
3814
3815
3816
3817
3818
3819
3820
3821
3822
3823
3824
3825
3826
3827
3828
3829
3830
3831
3832
3833
3834
3835
3836
3837
3838
3839
3840
3841
3842
3843
3844
3845
3846
3847
3848
3849
3850
3851
3852
3853
3854
3855
3856
3857
3858
3859
3860
3861
3862
3863
3864
3865
3866
3867
3868
3869
3870


5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357
5358
5359
5360
5361
5362
5363
5364
5365
5366
5367
5368
5369
5370
5371
5372
5373
5374
5375
5376
5377
5378
5379
5380
5381
5382
5383
5384
5385
5386
5387
5388
5389
5390
5391
5392
5393
5394
5395
5396
5397
5398
5399
5400
5401
5402
5403
5404
5405
5406
5407
5408
5409
5410
5411
5412
5413
5414
5415
5416
5417
5418
5419
5420
5421
5422
5423
5424
5425
5426
5427
5428
5429
5430
5431
5432
5433
5434
5435
5436
5437
5438
5439
5440
5441
5442
5443
5444
5445
5446
5447
5448
5449
5450
5451
5452
5453
5454
5455
5456
5457
5458
5459
5460
5461
5462
5463
5464
5465
5466
5467
5468
5469
5470
5471
5472
5473
5474
5475
5476
5477
5478
5479
5480
5481
5482
5483
5484
5485
5486
5487
5488
5489
5490
5491
5492
5493
5494
5495
5496
5497
5498
5499
5500
5501
5502
5503
5504
5505
5506
5507
5508
5509
5510
5511
5512
5513
5514
5515
5516
5517
5518
5519
5520
5521
5522
5523
5524
5525
5526
5527
5528
5529
5530
5531
5532
5533
5534
5535
5536


7162
7163
7164
7165
7166
7167
7168
7169
7170
7171
7172
7173
7174
7175
7176
7177
7178
7179
7180
7181
7182
7183
7184
7185
7186
7187
7188
7189
7190
7191
7192
7193
7194
7195
7196
7197
7198
7199
7200
7201
7202
7203
7204
7205
7206
7207
7208
7209
7210
7211
7212
7213
7214
7215
7216
7217
7218
7219
7220
7221
7222
7223
7224
7225
7226
7227
7228
7229
7230
7231
7232
7233
7234
7235
7236
7237
7238
7239
7240
7241
7242
7243
7244
7245
7246
7247
7248
7249
7250
7251
7252
7253
7254
7255
7256
7257
7258
7259
7260
7261
7262
7263
7264
7265
7266
7267
7268
7269
7270
7271
7272
7273
7274
7275
7276
7277
7278
7279
7280
7281
7282
7283
7284
7285
7286
7287
7288
7289
7290
7291
7292
7293
7294
7295
7296
7297
7298
7299
7300
7301
7302
7303
7304
7305
7306
7307
7308
7309
7310
7311
7312
7313
7314
7315
7316
7317
7318
7319
7320
7321
7322
7323
7324
7325
7326
7327
7328
7329
7330
7331
7332
7333
7334
7335
7336
7337
7338
7339
7340
7341
7342
7343
7344
7345
7346
7347
7348
7349
7350
7351
7352
7353
7354
7355
7356
7357
7358
7359
7360
7361


...

24064
24065
24066
24067
24068
24069
24070
24071
24072
24073
24074
24075
24076
24077
24078
24079
24080
24081
24082
24083
24084
24085
24086
24087
24088
24089
24090
24091
24092
24093
24094
24095
24096
24097
24098
24099
24100
24101
24102
24103
24104
24105
24106
24107
24108
24109
24110
24111
24112
24113
24114
24115
24116
24117
24118
24119
24120
24121
24122
24123
24124
24125
24126
24127
24128
24129
24130
24131
24132
24133
24134
24135
24136
24137
24138
24139
24140
24141
24142
24143
24144
24145
24146
24147
24148
24149
24150
24151
24152
24153
24154
24155
24156
24157
24158
24159
24160
24161
24162
24163
24164
24165
24166
24167
24168
24169
24170
24171
24172
24173
24174
24175
24176
24177
24178
24179
24180
24181
24182
24183
24184
24185
24186
24187
24188
24189
24190
24191
24192
24193
24194
24195
24196
24197
24198
24199
24200
24201
24202
24203
24204
24205
24206
24207
24208
24209
24210
24211
24212
24213
24214
24215
24216
24217
24218
24219
24220
24221
24222
24223
24224
24225
24226
24227
24228
24229
2423

25621
25622
25623
25624
25625
25626
25627
25628
25629
25630
25631
25632
25633
25634
25635
25636
25637
25638
25639
25640
25641
25642
25643
25644
25645
25646
25647
25648
25649
25650
25651
25652
25653
25654
25655
25656
25657
25658
25659
25660
25661
25662
25663
25664
25665
25666
25667
25668
25669
25670
25671
25672
25673
25674
25675
25676
25677
25678
25679
25680
25681
25682
25683
25684
25685
25686
25687
25688
25689
25690
25691
25692
25693
25694
25695
25696
25697
25698
25699
25700
25701
25702
25703
25704
25705
25706
25707
25708
25709
25710
25711
25712
25713
25714
25715
25716
25717
25718
25719
25720
25721
25722
25723
25724
25725
25726
25727
25728
25729
25730
25731
25732
25733
25734
25735
25736
25737
25738
25739
25740
25741
25742
25743
25744
25745
25746
25747
25748
25749
25750
25751
25752
25753
25754
25755
25756
25757
25758
25759
25760
25761
25762
25763
25764
25765
25766
25767
25768
25769
25770
25771
25772
25773
25774
25775
25776
25777
25778
25779
25780
25781
25782
25783
25784
25785
25786
2578

27119
27120
27121
27122
27123
27124
27125
27126
27127
27128
27129
27130
27131
27132
27133
27134
27135
27136
27137
27138
27139
27140
27141
27142
27143
27144
27145
27146
27147
27148
27149
27150
27151
27152
27153
27154
27155
27156
27157
27158
27159
27160
27161
27162
27163
27164
27165
27166
27167
27168
27169
27170
27171
27172
27173
27174
27175
27176
27177
27178
27179
27180
27181
27182
27183
27184
27185
27186
27187
27188
27189
27190
27191
27192
27193
27194
27195
27196
27197
27198
27199
27200
27201
27202
27203
27204
27205
27206
27207
27208
27209
27210
27211
27212
27213
27214
27215
27216
27217
27218
27219
27220
27221
27222
27223
27224
27225
27226
27227
27228
27229
27230
27231
27232
27233
27234
27235
27236
27237
27238
27239
27240
27241
27242
27243
27244
27245
27246
27247
27248
27249
27250
27251
27252
27253
27254
27255
27256
27257
27258
27259
27260
27261
27262
27263
27264
27265
27266
27267
27268
27269
27270
27271
27272
27273
27274
27275
27276
27277
27278
27279
27280
27281
27282
27283
27284
2728

28617
28618
28619
28620
28621
28622
28623
28624
28625
28626
28627
28628
28629
28630
28631
28632
28633
28634
28635
28636
28637
28638
28639
28640
28641
28642
28643
28644
28645
28646
28647
28648
28649
28650
28651
28652
28653
28654
28655
28656
28657
28658
28659
28660
28661
28662
28663
28664
28665
28666
28667
28668
28669
28670
28671
28672
28673
28674
28675
28676
28677
28678
28679
28680
28681
28682
28683
28684
28685
28686
28687
28688
28689
28690
28691
28692
28693
28694
28695
28696
28697
28698
28699
28700
28701
28702
28703
28704
28705
28706
28707
28708
28709
28710
28711
28712
28713
28714
28715
28716
28717
28718
28719
28720
28721
28722
28723
28724
28725
28726
28727
28728
28729
28730
28731
28732
28733
28734
28735
28736
28737
28738
28739
28740
28741
28742
28743
28744
28745
28746
28747
28748
28749
28750
28751
28752
28753
28754
28755
28756
28757
28758
28759
28760
28761
28762
28763
28764
28765
28766
28767
28768
28769
28770
28771
28772
28773
28774
28775
28776
28777
28778
28779
28780
28781
28782
2878

30115
30116
30117
30118
30119
30120
30121
30122
30123
30124
30125
30126
30127
30128
30129
30130
30131
30132
30133
30134
30135
30136
30137
30138
30139
30140
30141
30142
30143
30144
30145
30146
30147
30148
30149
30150
30151
30152
30153
30154
30155
30156
30157
30158
30159
30160
30161
30162
30163
30164
30165
30166
30167
30168
30169
30170
30171
30172
30173
30174
30175
30176
30177
30178
30179
30180
30181
30182
30183
30184
30185
30186
30187
30188
30189
30190
30191
30192
30193
30194
30195
30196
30197
30198
30199
30200
30201
30202
30203
30204
30205
30206
30207
30208
30209
30210
30211
30212
30213
30214
30215
30216
30217
30218
30219
30220
30221
30222
30223
30224
30225
30226
30227
30228
30229
30230
30231
30232
30233
30234
30235
30236
30237
30238
30239
30240
30241
30242
30243
30244
30245
30246
30247
30248
30249
30250
30251
30252
30253
30254
30255
30256
30257
30258
30259
30260
30261
30262
30263
30264
30265
30266
30267
30268
30269
30270
30271
30272
30273
30274
30275
30276
30277
30278
30279
30280
3028

In [64]:
# Теперь это массив сырых признаков (в строковом представлении, без отсева) #
features_list = np.array([np.array(line) for line in features_list])

In [65]:
features_list

array([['PREP', None, 'NOUN', ..., 'плата', '', '2'],
       ['NOUN', 'PREP', None, ..., '2', '', 'и'],
       [None, 'NOUN', 'CONJ', ..., 'и', 'в', 'у'],
       ..., 
       ['ADVB', 'NOUN', 'PREP', ..., 'к', '2', '.'],
       ['PREP', 'ADVB', None, ..., '.', 'метр', None],
       [None, 'PREP', None, ..., None, 'далее', None]], dtype=object)

In [75]:
# Заменяет лейбл на "*", если он "редкий" #
NUMBER_OF_OCCURENCES = 5
def get_feature(f, feature):
    if feature in _counters[f].keys() and _counters[f][feature] > NUMBER_OF_OCCURENCES:
        return feature
    else:
        return "*"

In [79]:
# Выкинем из этого массива классы, встретившиеся менее NUMBER_OF_OCCURENCES раз #
# Посчитаем частоту лейблов в столбце #
_number_of_columns = features_list.shape[1]
for u in range(_number_of_columns):
    arr = features_list[:, u]
    counter = Counter(arr)
    _counters.append(counter)

In [80]:
# Избавимся от редких лейблов (частота < NUMBER_OF_OCC) #
for y in range(len(features_list)):
    for x in range(_number_of_columns):
        features_list[y][x] = get_feature(x, features_list[y][x])

In [83]:
# Переводит категории в числовое представление #
class ColumnApplier(object):
    def __init__(self, column_stages):
        self._column_stages = column_stages

    def fit(self, x, y):
        for i, k in self._column_stages.items():
            k.fit(x[:, i])
        return self

    def transform(self, x):
        x = x.copy()
        for i, k in self._column_stages.items():
            x[:, i] = k.transform(x[:, i])
        return x

TypeError: list() takes at most 1 argument (3 given)

In [81]:
# Оставшиеся признаки бинаризуем #
_multi_encoder = ColumnApplier(dict([(i, preprocessing.LabelEncoder()) for i in range(len(features_list[0]))]))
features_list = _multi_encoder.fit(features_list, None).transform(features_list)
_enc = preprocessing.OneHotEncoder(dtype=np.bool_, sparse=True)
_enc.fit(features_list)
features_list = _enc.transform(features_list)

NameError: name 'ColumnApplier' is not defined

In [77]:
features_list

array([['PREP', None, 'NOUN', ..., 'плата', '*', '2'],
       ['NOUN', 'PREP', None, ..., '2', '*', 'и'],
       [None, 'NOUN', 'CONJ', ..., 'и', 'в', 'у'],
       ..., 
       ['ADVB', 'NOUN', 'PREP', ..., 'к', '2', '.'],
       ['PREP', 'ADVB', None, ..., '.', 'метр', '*'],
       [None, 'PREP', None, ..., '*', 'далее', '*']], dtype=object)

In [78]:
# Оставшиеся признаки бинаризуем #
features_list = _multi_encoder.transform(features_list)
features_list = _enc.transform(features_list)

NameError: name '_multi_encoder' is not defined

In [49]:
data

[[''],
 [''],
 'В',
 'понедельник',
 '28',
 'июня',
 'у',
 'здания',
 'мэрии',
 'Москвы',
 'на',
 'Тверской',
 'площади',
 'состоялась',
 'очередная',
 'несанкционированная',
 'акция',
 'протеста',
 '«',
 'День',
 'гнева',
 '»',
 ',',
 'в',
 'этот',
 'раз',
 'направленная',
 ',',
 'главным',
 'образом',
 ',',
 'против',
 'политики',
 'московских',
 'и',
 'подмосковных',
 'властей',
 '.',
 'Среди',
 'требований',
 ',',
 'выдвигаемых',
 'организаторами',
 'акции',
 ':',
 '«',
 'немедленная',
 'отставка',
 'мэра',
 'Москвы',
 'Юрия',
 'Лужкова',
 ',',
 'расследование',
 'итогов',
 'его',
 'деятельности',
 '»',
 ',',
 '«',
 'созыв',
 'московского',
 'общественного',
 'форума',
 'для',
 'обсуждения',
 'путей',
 'реформирования',
 'основных',
 'сфер',
 'жизнедеятельности',
 'в',
 'Москве',
 '»',
 ',',
 '«',
 'восстановление',
 'прямых',
 'выборов',
 'глав',
 'регионов',
 'России',
 '»',
 ',',
 '«',
 'роспуск',
 'нелегитимной',
 'Мосгордумы',
 '»',
 ',',
 'отставка',
 'подмосковного',
 'губер

In [55]:
X_train = fit_transform([[el] for el in factrueval_devset.words()], 
                            Y_train, 
                            path=TRAINSET_PATH)

NameError: name 'fit_transform' is not defined